## Oil and Gas Production Forecast
### Part_3.
### 1.1 Import Librries

In [11]:
import pandas as pd
import numpy as np
import warnings

from  datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import category_encoders as ce
import matplotlib.pyplot as plt
import gc
%matplotlib inline
warnings.filterwarnings('ignore')

### 1.2 Read in data

In [46]:
fld = pd.read_csv('data/fld.csv')

In [12]:
df_1 = pd.read_csv('data/df_1.csv')
df_2 = pd.read_csv('data/df_2.csv')
df_3 = pd.read_csv('data/df_3.csv')
df = df_1.append(df_2).append(df_3)
del df_1, df_2, df_3
gc.collect()

19

In [19]:
df.reset_index(drop=True, inplace=True)

In [21]:
df.head(4).append(df.tail(3))

,API Well Number,Year,Operator,OIL (Bbls),GAS (Mcf),WATER (Bbls),Months in Production,Well Type
0,31009019120000,1988,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
1,31009019120000,1989,"Sager, Howard & Judith",0.0,500.0,0.0,0.0,GD
2,31009019120000,1990,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
3,31009019120000,1991,"Sager, Howard & Judith",0.0,500.0,0.0,12.0,GD
121155,31029255050000,2017,U. S. Energy Development Corporation,0.0,2126.0,42.0,7.0,Gas Development
121156,31029255050000,2018,U. S. Energy Development Corporation,0.0,318.0,45.0,7.0,Gas Development
121157,31029255050000,2019,U. S. Energy Development Corporation,0.0,2436.0,0.0,12.0,Gas Development


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121158 entries, 0 to 121157
Data columns (total 8 columns):
API Well Number         121158 non-null int64
Year                    121158 non-null int64
Operator                121158 non-null object
OIL (Bbls)              102545 non-null float64
GAS (Mcf)               120678 non-null float64
WATER (Bbls)            116452 non-null float64
Months in Production    120252 non-null float64
Well Type               121158 non-null object
dtypes: float64(4), int64(2), object(2)
memory usage: 7.4+ MB


In [23]:
fld.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4717 entries, 0 to 4716
Data columns (total 51 columns):
API Well Number                   4717 non-null int64
County Code                       4717 non-null int64
API Hole Number                   4717 non-null int64
Sidetrack                         4717 non-null int64
Completion                        4717 non-null int64
Well Name                         4717 non-null object
Company Name                      4689 non-null object
Operator Number                   4717 non-null int64
Well Type                         4717 non-null object
Map Symbol                        4717 non-null object
Well Status                       4717 non-null object
Status Date                       572 non-null object
Permit Application Date           4413 non-null object
Permit Issued Date                4455 non-null object
Date Spudded                      4629 non-null object
Date of Total Depth               4460 non-null object
Date Well Completed 

In [24]:
fld.sample(3)

,API Well Number,County Code,API Hole Number,Sidetrack,Completion,Well Name,Company Name,Operator Number,Well Type,Map Symbol,...,Permit Fee,Objective Formation,Depth Fee,Spacing,Spacing Acres,Integration,Hearing Date,Date Last Modified,DEC Database Link,Location 1
2247,31013153710000,13,15371,0,0,"Trejchel, W 1",Empire Energy E&P LLC,2544,GD,GW,...,20.0,NaN,0.0,NaN,NaN,NaN,NaN,2010-06-04T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.52135', 'needs_recoding': F..."
3895,31013211590000,13,21159,0,0,Rammelt 3,Chautauqua Energy Inc.,654,GD,GW,...,725.0,Medina,625.0,NaN,NaN,NaN,NaN,2004-01-05T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.63338', 'needs_recoding': F..."
2891,31013177450000,13,17745,0,0,"Van Earden, J 1",N.E.A. Cross of N.Y. Inc.,1007,GD,GWP,...,1225.0,Medina,1125.0,NaN,NaN,NaN,NaN,2003-01-16T00:00:00,{'url': 'http://www.dec.ny.gov/cfmx/extapps/Ga...,"{'longitude': '-79.68321', 'needs_recoding': F..."


In [39]:
fld['Well Type'].unique()

array(['GD'], dtype=object)

In [40]:
fld['Map Symbol'].unique()

array(['GWP', 'GW', 'O'], dtype=object)

In [30]:
fld.describe()

,API Well Number,County Code,API Hole Number,Sidetrack,Completion,Operator Number,Region,"Proposed Depth, ft",Surface Longitude,Surface Latitude,Bottom Hole Longitude,Bottom Hole Latitude,"True Vertical Depth, ft","Measured Depth, ft","Kickoff, ft","Drilled Depth, ft","Elevation, ft",Permit Fee,Depth Fee,Spacing Acres
count,4.717000e+03,4717.000000,4717.000000,4717.0,4717.000000,4717.000000,4717.0,4716.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.000000,4717.0,4717.000000,4444.000000,4716.000000,4717.000000,22.000000
mean,3.101271e+13,12.544626,16953.453890,0.0,0.018868,2272.140767,9.0,2812.433418,-79.316538,42.258476,-79.316536,42.258484,3554.433326,3554.430570,0.0,3554.430570,1369.596985,611.036684,559.059360,53.965909
std,1.620988e+09,1.622432,6011.530453,0.0,0.146576,1527.626292,0.0,1690.720419,0.222828,0.143486,0.222827,0.143491,775.597958,775.597873,0.0,775.597873,345.668537,656.604968,612.319424,27.441319
min,3.100901e+13,9.000000,255.000000,0.0,0.000000,13.000000,9.0,0.000000,-79.761600,41.999480,-79.761600,41.999480,0.000000,0.000000,0.0,0.000000,3.000000,0.000000,0.000000,35.200000
25%,3.101311e+13,13.000000,13043.000000,0.0,0.000000,1192.000000,9.0,2100.000000,-79.509110,42.151120,-79.509110,42.151120,3116.000000,3116.000000,0.0,3116.000000,1260.000000,0.000000,0.000000,40.000000
50%,3.101316e+13,13.000000,17148.000000,0.0,0.000000,2544.000000,9.0,3520.000000,-79.293470,42.257070,-79.293470,42.257070,3708.000000,3708.000000,0.0,3708.000000,1461.000000,20.000000,0.000000,41.030000
75%,3.101319e+13,13.000000,20246.000000,0.0,0.000000,2544.000000,9.0,4095.000000,-79.139240,42.369260,-79.139240,42.369260,4166.000000,4166.000000,0.0,4166.000000,1610.000000,1225.000000,1125.000000,51.130000
max,3.102966e+13,29.000000,67321.000000,0.0,2.000000,9998.000000,9.0,6501.000000,-78.756970,42.693560,-78.756970,42.693560,6182.000000,6182.000000,0.0,6182.000000,2625.000000,11000.000000,10900.000000,146.800000


In [35]:
fld['Completion'].value_counts()

0    4635
1      75
2       7
Name: Completion, dtype: int64

In [44]:
# fld['Spacing Acres'].unique()

In [50]:
# fld['Location 1'].unique()

In [52]:
# fld['Surface Latitude'].unique()
fld['Original Well Type'].unique()

array(['NL', 'GW', 'GD', 'OD'], dtype=object)

In [56]:
# Drop Numerical columns which are not required for analysis
fld.drop(columns=['API Hole Number','Sidetrack','Permit Fee', 'Depth Fee', 'Spacing Acres'], inplace=True)

In [58]:
# Drop Categorical columns which are not required for analysis
fld.drop(columns=['Financial Security','Location 1', 'Integration', 'Hearing Date', 'Date Last Modified', 'DEC Database Link', 'Spacing', 'State Lease'], inplace=True)

In [61]:
del fld['Well Type']

In [62]:
fld

,API Well Number,County Code,Completion,Well Name,Company Name,Operator Number,Map Symbol,Well Status,Status Date,Permit Application Date,...,Surface Latitude,Bottom Hole Longitude,Bottom Hole Latitude,"True Vertical Depth, ft","Measured Depth, ft","Kickoff, ft","Drilled Depth, ft","Elevation, ft",Original Well Type,Objective Formation
0,31009008540000,9,0,Marsh William 1,Belmont Quadrangle Drilling,9034,GWP,PA,NaN,NaN,...,42.281410,-79.025660,42.281410,3620.0,3620.0,0.0,3620.0,1367.0,NL,NaN
1,31009008550000,9,0,Greely Clyde 1,Belmont Quadrangle Drilling,9034,GWP,PA,NaN,NaN,...,42.291590,-79.009200,42.291590,3562.0,3562.0,0.0,3562.0,1344.0,NL,NaN
2,31009008580000,9,0,Franklin G M 1,Belmont Quadrangle Drilling,9034,GWP,PA,NaN,NaN,...,42.291450,-79.024190,42.291450,3717.0,3717.0,0.0,3717.0,1526.0,NL,NaN
3,31009008590000,9,0,Curtiss P H 1,Belmont Quadrangle Drilling,9034,GWP,PA,NaN,NaN,...,42.289390,-79.017160,42.289390,3558.0,3558.0,0.0,3558.0,1345.0,NL,NaN
4,31009008600000,9,0,Brown A J 1,J. J. Bucher Producing Corporation,154,GWP,PA,NaN,NaN,...,42.287460,-79.023450,42.287460,3587.0,3587.0,0.0,3587.0,1364.0,NL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4712,31029254720000,29,0,Catalano 1,U. S. Energy Development Corporation,708,GW,AC,NaN,2008-04-22T00:00:00,...,42.594072,-79.004533,42.594072,2005.0,2005.0,0.0,2005.0,728.0,GD,Medina
4713,31029255050000,29,0,Catalano 2,U. S. Energy Development Corporation,708,GW,AC,NaN,2008-05-13T00:00:00,...,42.590258,-79.002178,42.590258,2041.0,2041.0,0.0,2041.0,743.0,GD,Medina
4714,31029557810000,29,0,Woods Parcel 2,The Woods Parcel LLC,2470,GWP,PA,2009-02-25T00:00:00,NaN,...,42.693560,-79.037890,42.693560,1500.0,1500.0,0.0,1500.0,600.0,NL,Medina
4715,31029664360000,29,0,Hoag 1660-I,Seneca Nation of Indians,9006,GW,UN,NaN,NaN,...,42.518280,-78.962180,42.518280,2307.0,2307.0,0.0,2307.0,NaN,GD,Medina
